In [ ]:
!pip install keras==2.8.0
!pip install tensorflow==2.8.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 11.3 MB/s eta 0:00:00
  Attempting uninstall: keras
    Found existing installation: keras 2.12.0
    Uninstalling keras-2.12.0:
      Successfully uninstalled keras-2.12.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.12.0 requires keras<2.13,>=2.12.0, but you have keras 2.8.0 which is incompatible.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 497.6/497.6 MB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 28.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 462.5/462.5 kB 15.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.9/4.9 MB 22.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 781.3/781.3 kB 29.7 MB/s eta 0:00:00
  Attempting unins

In [ ]:
from sklearn.metrics import recall_score, precision_score, f1_score, roc_auc_score, accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeClassifier
from collections import defaultdict
from warnings import filterwarnings
from scipy.special import softmax
from xgboost import XGBClassifier
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
from keras import backend as K
from collections import deque
from typing import Callable
from sklearn import svm
import tensorflow as tf
from typing import List
import seaborn as sns
import pandas as pd
import numpy as np
import requests
import logging
import random
import keras
import json
import copy
import os
logging.getLogger("tensorflow").setLevel(logging.ERROR)
sns.set(rc = {'figure.figsize':(22,12)}, style="whitegrid")
filterwarnings('ignore')

In [ ]:
from google.colab import drive
drive.mount("/content/drive")
data_path = "/content/drive/MyDrive/Colab Notebooks/Muawiya/Js_Contana/dynamic-feature-selection/Data/isam data"
root_reslts_path = "/content/drive/MyDrive/Colab Notebooks/Muawiya/Js_Contana/dynamic-feature-selection/Results"

Mounted at /content/drive


In [ ]:
def save_object(obj, filename,path):
    filename = os.path.join(path,filename)
    with open(filename+".pkl", 'wb') as outp:
        pickle.dump(obj, outp, pickle.HIGHEST_PROTOCOL)
    outp.close()
def load_object(filename,path):
    filename = os.path.join(path,filename)
    with open(filename+".pkl", 'rb') as outp:
        loaded_object = pickle.load(outp)
    outp.close()
    return loaded_object

# 1. Policy Function & DQN Architicture
* 1- epsilon greedy implementaion for make action
* 2- DQN model and compile

In [ ]:
def epsilon_greedy(expected_reward, epsilon=0.97) -> int:
    """
    expected_reward: list of expected rewards for each possible action
    epsilon: .
    """
    if np.random.rand() <= epsilon:
        return np.random.choice(list(range(len(expected_reward))))
    else:
        return np.argmax(expected_reward)

PolicyFunction  = Callable[[np.ndarray, float], int]

In [ ]:
LEARNING_RATE = 0.001

def create_model(input_dim):
    K.clear_session()
    model = keras.models.Sequential()
    model.add(keras.layers.Input(shape=(input_dim,)))
    model.add(keras.layers.Dense(32, kernel_initializer='he_uniform', activation='relu'))
    model.add(keras.layers.Dense(16, kernel_initializer='he_uniform', activation='relu'))
    model.add(keras.layers.Dense(2))
    model.compile(loss='mse', optimizer='adam')
    return model

## 2. Agents Implementaion

* Agents class will be the parent of 4 types of agents listed as follow:
    * 1- Softmax version (distrbute the total reward between agents using softmax function)
    * 2- Average version (distrbute the total reward between agents using average function)
    * 1- regression version (calcualte the contrbution of each agent using regression model)
    * 1- Single Agent version (only one agent at a time can make action and get the total reawrd as a result)

In [ ]:
class Agents:
    # class variable
    agent_count = 0

    def __init__(self, evaluation_network, buffer_size: int = 800):

        self.evaluation_network = evaluation_network
        self.target_network = copy.deepcopy(self.evaluation_network)
        self.buffer_size = buffer_size
        self.fitted = False
        self.agent_id = Agents.agent_count
        Agents.agent_count += 1
        # reply buffer is a list of tuples each tuples contains the following
        # (St, At, St+1, Rt+1)
        # (Current state, Action was made, New state, Reward)
        self.reply_buffer = deque(maxlen=self.buffer_size)
        self.contrbution = np.random.rand()

    def make_action(self, curr_state: np.ndarray, policy_function: PolicyFunction, epsilon) -> int:
        # q_values represents the expected rewards for each possible action
        if self.fitted:
            q_values = self.evaluation_network.predict(curr_state.reshape(-1, Agents.agent_count))
            action = policy_function(q_values, epsilon)
        else:
            action = policy_function([0, 1], 1)
        return action

    def update_target_network(self):
        self.target_network = copy.deepcopy(self.evaluation_network)
        return

In [ ]:
class AgentsSoftmax(Agents):
    def __init__(self, evaluation_network, buffer_size=800):
        super().__init__(evaluation_network, buffer_size)


    def update_evaluation_network(self, batch_size=32, epochs=5, discount_factor=0.995):
        # select random batch from the reply buffer
        batch = random.sample(self.reply_buffer, batch_size)

        # inintilize some lists to store transition information
        Q1, actions, Q2, rewards = [], [], [], []

        # from each transition extract its values
        for transition in batch:
            Q1.append(transition[0])
            actions.append(transition[1])
            Q2.append(transition[2])
            rewards.append(transition[3])
        # X_train will be the states from
        X_train = np.array(Q1).reshape(-1, Agents.agent_count)


        expected_reward = self.evaluation_network.predict(np.squeeze(Q1).reshape(-1, Agents.agent_count))
        Q2 = self.target_network.predict(np.squeeze(Q1).reshape(-1, Agents.agent_count))

        # update expected rewards using biliman equation

        for i, act in enumerate(actions[:-1]):
            expected_reward[i, act] = rewards[i] + (discount_factor * np.argmax(Q2[i]))

        y_train = expected_reward.copy()

        # calculate the change frequency of the agent decision to use it as its contrbution in get total reward

        change_frequency = 0
        for state, next_state, reward, next_reward in zip(X_train[:-1], X_train[1:], rewards[: -1], rewards[1:]):
            #print(state, next_state, reward, next_reward)
            if np.abs(state[self.agent_id] - next_state[self.agent_id]) == 1:
                self.contrbution += np.abs(reward - next_reward)
                change_frequency += 1
        self.contrbution /= change_frequency

        # train the DQN evaluation network.
        self.evaluation_network.fit(X_train, y_train, epochs=epochs, verbose=0)
        self.fitted = True
        return

In [ ]:
class AgentsRegression(Agents):
    def __init__(self, evaluation_network, buffer_size=800):
        super().__init__(evaluation_network, buffer_size)

    def update_evaluation_network(self, batch_size=32, epochs=5, discount_factor=0.995):
        # select random batch from the reply buffer
        batch = random.sample(self.reply_buffer, batch_size)

        # inintilize some lists to store transition information
        Q1, actions, Q2, rewards = [], [], [], []

        # from each transition extract its values
        for transition in batch:
            Q1.append(transition[0])
            actions.append(transition[1])
            Q2.append(transition[2])
            rewards.append(transition[3])
        # X_train will be the states from
        X_train = np.array(Q1).reshape(-1, Agents.agent_count)

        expected_reward = self.evaluation_network.predict(np.array(Q1).reshape(-1, Agents.agent_count))
        Q2 = self.target_network.predict(np.array(Q2).reshape(-1, Agents.agent_count))

        # update expected rewards using biliman equation
        for i, act in enumerate(actions[:-1]):
            expected_reward[i, act] = rewards[i] + (discount_factor * np.argmax(Q2[i]))

        y_train = expected_reward.copy()
        # train the DQN evaluation network.
        self.evaluation_network.fit(X_train, y_train, epochs=epochs, verbose=0)
        self.fitted = True
        return

In [ ]:
class AgentsAverage(Agents):
    def __init__(self, evaluation_network, buffer_size=800):
        super().__init__(evaluation_network, buffer_size)

    def update_evaluation_network(self, batch_size=32, epochs=5, discount_factor=0.995):
        # select random batch from the reply buffer
        batch = random.sample(self.reply_buffer, batch_size)

        # inintilize some lists to store transition information
        Q1, actions, Q2, rewards = [], [], [], []

        # from each transition extract its values
        for transition in batch:
            Q1.append(transition[0])
            actions.append(transition[1])
            Q2.append(transition[2])
            rewards.append(transition[3])
        # X_train will be the states from
        X_train = np.array(Q1).reshape(-1, Agents.agent_count)

        expected_reward = self.evaluation_network.predict(np.array(Q1).reshape(-1, Agents.agent_count))
        Q2 = self.target_network.predict(np.array(Q2).reshape(-1, Agents.agent_count))

        # update expected rewards using biliman equation

        for i, act in enumerate(actions[:-1]):
            expected_reward[i, act] = rewards[i] + (discount_factor * np.argmax(Q2[i]))

        y_train = expected_reward.copy()

        WINDOW_SIZE = 4
        X_train_ = np.zeros((X_train.shape[0] // WINDOW_SIZE, X_train.shape[1]))
        y_train_ = []
        j = 0
        for i in range(0, batch_size, WINDOW_SIZE):
            window_of_states = X_train[i: i + WINDOW_SIZE].sum(axis=0) / WINDOW_SIZE
            window_of_rewards = sum(rewards[i: i + WINDOW_SIZE])
            r = window_of_rewards * window_of_states[self.agent_id]
             # Rounding state
            X_train_[j, :] = np.around(window_of_states)
            if window_of_states[self.agent_id] == 0:
                if window_of_rewards > 0.6:
                    r = window_of_rewards
                else:
                    r = window_of_rewards / WINDOW_SIZE
            y_train_.append(r)
            j += 1

        X_train = X_train_
        y_train = np.array(y_train_)
        # train the DQN evaluation network.
        self.evaluation_network.fit(X_train, y_train, epochs=epochs, verbose=0)
        self.fitted = True
        return

In [ ]:
class AgentsSingle(Agents):
    def __init__(self, evaluation_network, buffer_size=800):
        super().__init__(evaluation_network, buffer_size)

    def update_evaluation_network(self, batch_size=32, epochs=5, discount_factor=0.995):
        # select random batch from the reply buffer
        batch = random.sample(self.reply_buffer, batch_size)

        # inintilize some lists to store transition information
        Q1, actions, Q2, rewards = [], [], [], []

        # from each transition extract its values
        for transition in batch:
            Q1.append(transition[0])
            actions.append(transition[1])
            Q2.append(transition[2])
            rewards.append(transition[3])
        # X_train will be the states from
        X_train = np.array(Q1).reshape(-1, Agents.agent_count)

        expected_reward = self.evaluation_network.predict(np.array(Q1).reshape(-1, Agents.agent_count),verbose=0)
        Q2 = self.target_network.predict(np.array(Q2).reshape(-1, Agents.agent_count), verbose=0)

        # update expected rewards using biliman equation
        for i, act in enumerate(actions[:-1]):
            expected_reward[i, act] = rewards[i] + (discount_factor * np.argmax(Q2[i]))

        y_train = expected_reward.copy()
        # train the DQN evaluation network.
        self.evaluation_network.fit(X_train, y_train, epochs=epochs, verbose=0)
        self.fitted = True
        return

# 3. Reward Calculation Method
* 1- get accuracy of selected feature using logistic regression model
* 2- claculate the reward with reward_strategy function using accuracy from last step.

In [ ]:
# test set percentage
TESTSIZE=0.2

def get_reward(dataset:pd.DataFrame, subset_features:list, y_label:str):
    global TESTSIZE
    # index of selected features
    subset_features = np.where(np.array(subset_features) == 1)[0]
    if subset_features.shape[0] == 0:return 0
    # train test split
    X, Y = dataset[subset_features.tolist()], dataset[y_label]
    X_train, X_test, y_train, y_test = train_test_split(X, Y, random_state=42, test_size=TESTSIZE)
    # data normalization.
    std = StandardScaler()
    X_train = std.fit_transform(X_train)
    X_test = std.transform(X_test)
    # classification and model evaluation
    classifier = LogisticRegression(max_iter=500)
    classifier.fit(X_train, y_train)
    y_pred = classifier.predict(X_test)
    acc = accuracy_score(y_test, y_pred)
    return acc

In [ ]:
def reward_strategy(time_step: int, accuracy: float, accuracy_history: list, subset_features: list, error_rate: float,
                    beta: float = 0.99):
    if sum(subset_features) == len(subset_features):
        return -5
    elif accuracy > max(accuracy_history):
        return 0.5
    elif accuracy < max(accuracy_history):
        return -0.1
    else:
        return -1 * (beta * error_rate + ((1 - beta) * (sum(subset_features) / len(subset_features))))

# 4. Some helper function
* 1- Object to store all information about the result like dataset name, accuracy, precision and etc...
* 2- Telegram api to send the result to a chat room
* 3- Send result to the chat room

In [ ]:
import pickle

class Results:
    def __init__(self, method_name, dataset_name, chunk_id, feature_space):
        self.dataset_name = dataset_name
        self.method_name = method_name
        self.chunk_id = chunk_id
        self.feature_space = feature_space
        self.feature_space_size = len(feature_space)
        self.result_information = {}

    def set_chunk_id(self, chunk_id: int):
        self.chunk_id = chunk_id

    def set_feature_space(self, feature_space: list):
        self.feature_space = feature_space
        self.feature_space_size = len(feature_space)

    def add_result(self, model_type:str, result:dict):
        self.result_information[model_type] = result

    def save(self, path='feature_selection_results'):
        file_name = self.method_name + '_' + self.dataset_name + '_' + '{}'.format(self.chunk_id) + '.pkl'
        with open(os.path.join(path, file_name), 'wb') as file_:
            pickle.dump(self, file_, pickle.HIGHEST_PROTOCOL)
        return

In [ ]:
class Telegram:

    def __init__(self, bot_token):
        self.end_point = 'https://api.telegram.org/bot'
        self.token = bot_token
        self.full_endpoint = self.end_point + self.token + '/'

    def __repr__(self):
        return 'your token is {}'.format(self.full_endpoint)

    def send_message(self, chat_id, message):
        send_text = self.full_endpoint + 'sendMessage'
        data = {'chat_id': chat_id, 'text': message}
        response = requests.get(send_text, data=data)
        return response

    def send_photo(self, chat_id, photo):
        url = self.full_endpoint + 'sendPhoto'
        data = {'chat_id': chat_id}
        files = {'photo': open(photo, 'rb')}
        response = requests.post(url, data=data, files=files)
        return response

    def send_document(self, chat_id, file):
        url = self.full_endpoint + 'sendDocument'
        data = {'chat_id': chat_id}
        files = {'document':open(file, 'rb')}
        response = requests.post(url, data = data, files = files)
        return response
    def get_updates(self):
        url = self.full_endpoint + 'getUpdates'
        response = requests.get(url)
        return response

    def get_file_information(self, file_id):
        url = f'https://api.telegram.org/bot{self.token}/getFile'
        response = requests.post(url,data = {"file_id":file_id})
        if response.status_code != 200:
            return {"ok":"False"}
        json_response = response.json()
        if json_response['ok'] == False:
            return {"ok":"False"}
        file_path = json_response['result']['file_path']
        file_information = requests.get(f'https://api.telegram.org/file/bot{self.token}/{file_path}')
        return file_information.text

In [ ]:
def send_results(telegram_api, results):
    telegram.send_message(1021388563, 'dataset name : {}'.format(results.dataset_name))
    telegram.send_message(1021388563, 'chunk id : {}'.format(results.chunk_id))
    telegram.send_message(1021388563, 'selected features : {}'.format(results.feature_space))
    telegram.send_message(1021388563, 'results')
    for key in  results.result_information.keys():
        telegram.send_message(1021388563, 'model tpye : {}'.format(key))
        telegram.send_message(1021388563, '{}'.format(results.result_information[key]))

# 5. Feature Selection Main Algorithm
* For each type of agents we need a specific feature selection algorithm with little difference between them.
* Evaluation function using different machine learning model based on selected feature
* Result Visulization

In [ ]:
def softmax_distrbution(agents):
    contrbutions = []
    for agent in agents:
        contrbutions.append(agent.contrbution)
    return softmax(contrbutions)


def random_forest_distrbution(dataset,num_of_agents, num_of_samples=10000):
    X = []
    y = []
    for _ in range(num_of_samples):
        features_space = np.random.choice([0, 1], size=(num_of_agents,)).tolist()
        accuracy = get_reward(dataset, features_space, 'target')
        X.append(features_space)
        y.append(accuracy)
    X = np.array(X)
    y = np.array(y)
    rf = RandomForestRegressor(n_estimators=15)
    rf.fit(X, y)
    return rf.feature_importances_.tolist()

In [ ]:
def feature_selection(algo_type:str, agents: list, dataset: pd.DataFrame, y_label: str, eposide=200):
    """
    """
    column_names = list(range(dataset.shape[1]))
    column_names[-1] = 'target'
    dataset.columns = column_names
    epsilon = 0.01
    features_space = []
    NUMBER_OF_AGENTS = dataset.shape[1] - 1

    # get contrbutions
    contrbutions = []
    if algo_type == 'random_forest':
        contrbutions = random_forest_distrbution(dataset,NUMBER_OF_AGENTS)
    elif algo_type in ['single_agent', 'average']:
        contrbutions = [1] * NUMBER_OF_AGENTS

    for i in tqdm(range(eposide)):
        # define the initial space
        features_space = np.random.choice([0, 1], size=(NUMBER_OF_AGENTS,)).tolist()
        # rewards history
        rewards = [0]

        # get action of each agent to create new feature space
        next_feature_space = []
        # contrbution of each agent

        if algo_type == 'softmax':
            contrbutions = softmax_distrbution(agents)

        for t in range(0, NUMBER_OF_AGENTS):
            action = agents[t].make_action(np.array(features_space.copy()), epsilon_greedy, epsilon)
            next_feature_space.append(action)
            if algo_type == 'single_agent':
                features_space[t] = action


        # calculate the total accuracy of new state (new feature space) and distrbute it using softmax

        # 1- get the accuracy using machine learning model trained in the current subset feature
        reward_as_accuracy = get_reward(dataset, next_feature_space, y_label)

        # 2- using the reward strategy map the accuracy value (reward_as_accuracy) to new reward value (reward_at_time_t)
        reward_at_time_t = reward_strategy(t, reward_as_accuracy, rewards, next_feature_space, 1 - reward_as_accuracy)
        # add the accuray of machine learning model to rewards list to use it in the mapping reward strategy.
        rewards.append(reward_as_accuracy)

        # total reward = reward after mapping
        total_reward = reward_at_time_t



        # add state and actions to agent buffer reply  and the reward which equals to contrbution of the agent*total reward
        transition = []
        for t in range(0, NUMBER_OF_AGENTS):
            transition.clear()
            # add current state (current feature space )
            feature_space_copy = features_space.copy()
            transition.append(feature_space_copy)
            # add agent's action to the transition
            action = next_feature_space[t]
            transition.append(action)
            # add new state (new feature space) into transition
            transition.append(next_feature_space)
            # add distrbuted reward to the transition
            transition.append(total_reward * contrbutions[t])
            # add the transition to reply buffer
            agents[t].reply_buffer.append(transition)
            if len(agents[t].reply_buffer) > 32 and i % 32 == 0:
                agents[t].update_evaluation_network()
        if i % 64 == 0:
            for agent in agents:
                if agent.fitted:
                    agent.update_target_network()
        epsilon = 0.97 * epsilon
    return next_feature_space

In [ ]:
def model_evaluation(method_name, dataset_name, dataset, selected_features, target_column, chunk_id, telegram_api=None):
    models = [
              ('SVM', svm.SVC(kernel='rbf', max_iter=8000, C=0.2, probability=True)),
              ('KNN', KNeighborsClassifier(5)),
              ('DecisionTree', DecisionTreeClassifier(random_state=42)),
              ('RandomForest', RandomForestClassifier()),
              ('LogRegression', LogisticRegression(max_iter=500))
     ]
    # index of selected features
    subset_features = np.where(np.array(selected_features) == 1)[0]
    if subset_features.shape[0] == 0:return 0
    # train test split
    X, Y = dataset[subset_features.tolist()], dataset[target_column]
    X_train, X_test, y_train, y_test = train_test_split(X, Y, random_state=42, test_size=TESTSIZE)
    # data normalization.
    std = StandardScaler()
    X_train = std.fit_transform(X_train)
    X_test = std.transform(X_test)
    # classification and model evaluation
    results = Results(method_name, dataset_name, chunk_id, subset_features.tolist())
    for model in models:
        model_name, model_obj = model[0], model[1]
        model_obj.fit(X_train, y_train)
        y_pred = model_obj.predict(X_test)
        acc, p, r, f1, auc = accuracy_score(y_test, y_pred), precision_score(y_test, y_pred), recall_score(y_test, y_pred), f1_score(y_test, y_pred), roc_auc_score(y_test, y_pred)
        results.add_result(model_name, {'accuracy': acc, 'precision': p, 'recall': r, 'f1-score': f1, 'AUC': auc})
    # send_results(telegram_api, results)
    # results.save()
    print(results.result_information)

In [ ]:
def main(filenames, algorithm_type, telegram_api=None):
    # store the results
    for f_name in filenames:
        # read csv file
        df = pd.read_csv(f_name)
        column_names = list(range(df.shape[1]))
        column_names[-1] = 'target'
        df.columns = column_names
        # number of row in the dataframe
        number_of_samples = df.shape[0]
        # define the cunck size
        CHUNK_SIZE = number_of_samples // 10
        #####################################################
        chunk_id = 1
        for i in range(0, df.shape[0], CHUNK_SIZE):
            if chunk_id > 9:
                break
            if chunk_id < 4 and algorithm_type=='random_forest':
              print("skip chunk_id : "+str(chunk_id))
              chunk_id+=1
              continue
            chunk = df.iloc[i: i + CHUNK_SIZE, :]
            X, Y = chunk.drop('target', axis=1), chunk['target']
            agents = []
            #AgentsRegression.agent_count = 0
            #AgentsSoftmax.agent_count = 0
            #AgentsSingle.agent_count = 0
            #AgentsAverage.agent_count = 0
            Agents.agent_count = 0
            NUM_OF_FEATURES = df.shape[1] - 1 #  -1 for the target column


            for i in range(NUM_OF_FEATURES):
                if algorithm_type == 'softmax':
                    agents.append(AgentsSoftmax(create_model(NUM_OF_FEATURES)))
                elif algorithm_type == 'average':
                    agents.append(AgentsAverage(create_model(NUM_OF_FEATURES)))
                elif algorithm_type == 'single_agent':
                    agents.append(AgentsSingle(create_model(NUM_OF_FEATURES)))
                elif algorithm_type == 'random_forest':
                    agents.append(AgentsRegression(create_model(NUM_OF_FEATURES)))

            result = feature_selection(algorithm_type, agents, chunk, 'target')
            save_object(result, algorithm_type+" mask for chunk number "+str(chunk_id),reslts_path)
            model_evaluation(algorithm_type, f_name, chunk, result, 'target', chunk_id, telegram_api=None)
            chunk_id += 1

In [ ]:
# IramTariq-DMOZ-xssed--> D4-30
# Data_167_featurs--> Mokbal-2021(D2-167)
# IramTariq-DataTakenFromYunZhou--> D3-30
# Data_66_featurs--> D1-N66
# telegram = Telegram('5328835953:AAEisAKmtBl9TfGOFHQW0KaXcynsdRo7OKE')
filenames = ['Data_66_featurs.csv']
filenames = list(map(lambda x: os.path.join(data_path, x), filenames))
reslts_path = os.path.join(root_reslts_path,filenames[0].split("/")[-1].split(".")[0])
filenames

['/content/drive/MyDrive/Colab Notebooks/Muawiya/Js_Contana/dynamic-feature-selection/Data/isam data/Data_66_featurs.csv']

In [ ]:
# indices_to_access = [2, 3, 6]

# accessed_mapping = map(filenames.__getitem__, indices_to_access)
# filenames = list(accessed_mapping)
# filenames

In [ ]:
import sys
for algo in ['softmax','average', 'single_agent','random_forest']:
    main(filenames, algo)

skip chunk_id : 1
skip chunk_id : 2
skip chunk_id : 3


  0%|          | 0/200 [00:00<?, ?it/s]

{'SVM': {'accuracy': 0.9393939393939394, 'precision': 0.9114114114114115, 'recall': 0.8477653631284916, 'f1-score': 0.8784370477568743, 'AUC': 0.9095344325370084}, 'KNN': {'accuracy': 0.9347041847041847, 'precision': 0.8928046989720999, 'recall': 0.8491620111731844, 'f1-score': 0.8704366499642091, 'AUC': 0.9068280872986544}, 'DecisionTree': {'accuracy': 0.9512987012987013, 'precision': 0.8963165075034106, 'recall': 0.9175977653631285, 'f1-score': 0.906832298136646, 'AUC': 0.9403163924091908}, 'RandomForest': {'accuracy': 0.9736652236652237, 'precision': 0.9521800281293952, 'recall': 0.9455307262569832, 'f1-score': 0.948843728100911, 'AUC': 0.9644968806382191}, 'LogRegression': {'accuracy': 0.9271284271284271, 'precision': 0.8681948424068768, 'recall': 0.8463687150837989, 'f1-score': 0.8571428571428572, 'AUC': 0.9008108166858684}}


  0%|          | 0/200 [00:00<?, ?it/s]

{'SVM': {'accuracy': 0.9613997113997114, 'precision': 0.9985096870342772, 'recall': 0.8634020618556701, 'f1-score': 0.9260539046302695, 'AUC': 0.931450529925831}, 'KNN': {'accuracy': 0.9754689754689755, 'precision': 0.9809782608695652, 'recall': 0.9304123711340206, 'f1-score': 0.955026455026455, 'AUC': 0.9616991715389542}, 'DecisionTree': {'accuracy': 0.9715007215007215, 'precision': 0.9520103761348897, 'recall': 0.9458762886597938, 'f1-score': 0.9489334195216548, 'AUC': 0.9636696072557485}, 'RandomForest': {'accuracy': 0.9862914862914863, 'precision': 0.9880952380952381, 'recall': 0.9626288659793815, 'f1-score': 0.975195822454308, 'AUC': 0.9790599239716546}, 'LogRegression': {'accuracy': 0.9693362193362194, 'precision': 0.9914651493598862, 'recall': 0.8981958762886598, 'f1-score': 0.942528735632184, 'AUC': 0.9475949321323058}}


...

{'SVM': {'accuracy': 0.9087301587301587, 'precision': 0.852054794520548, 'recall': 0.8109517601043025, 'f1-score': 0.8309953239812959, 'AUC': 0.8785432117229741}, 'KNN': {'accuracy': 0.9444444444444444, 'precision': 0.9158751696065129, 'recall': 0.8800521512385919, 'f1-score': 0.8976063829787233, 'AUC': 0.9245647289858795}, 'DecisionTree': {'accuracy': 0.9538239538239538, 'precision': 0.9209486166007905, 'recall': 0.9113428943937418, 'f1-score': 0.9161205766710353, 'AUC': 0.9407088536806614}, 'RandomForest': {'accuracy': 0.9751082251082251, 'precision': 0.958005249343832, 'recall': 0.9517601043024772, 'f1-score': 0.9548724656638325, 'AUC': 0.9679000022759268}, 'LogRegression': {'accuracy': 0.922077922077922, 'precision': 0.8728010825439784, 'recall': 0.8409387222946545, 'f1-score': 0.8565737051792829, 'AUC': 0.8970279646385991}}


  0%|          | 0/200 [00:00<?, ?it/s]

{'SVM': {'accuracy': 0.9678932178932179, 'precision': 0.9985486211901307, 'recall': 0.8865979381443299, 'f1-score': 0.9392491467576792, 'AUC': 0.943048468070161}, 'KNN': {'accuracy': 0.9787157287157288, 'precision': 0.9877551020408163, 'recall': 0.9355670103092784, 'f1-score': 0.9609530112508272, 'AUC': 0.965528996136603}, 'DecisionTree': {'accuracy': 0.9816017316017316, 'precision': 0.967741935483871, 'recall': 0.9664948453608248, 'f1-score': 0.9671179883945841, 'AUC': 0.9769848976303122}, 'RandomForest': {'accuracy': 0.9884559884559885, 'precision': 0.9920634920634921, 'recall': 0.9664948453608248, 'f1-score': 0.97911227154047, 'AUC': 0.9817444166683884}, 'LogRegression': {'accuracy': 0.9758297258297258, 'precision': 0.9903181189488244, 'recall': 0.9226804123711341, 'f1-score': 0.9553035356904603, 'AUC': 0.959586699171539}}


  0%|          | 0/200 [00:00<?, ?it/s]

{'SVM': {'accuracy': 0.9451659451659452, 'precision': 0.9407008086253369, 'recall': 0.8660049627791563, 'f1-score': 0.9018087855297157, 'AUC': 0.9218122474119586}, 'KNN': {'accuracy': 0.9502164502164502, 'precision': 0.9326424870466321, 'recall': 0.8933002481389578, 'f1-score': 0.9125475285171102, 'AUC': 0.9334253020959286}, 'DecisionTree': {'accuracy': 0.9386724386724387, 'precision': 0.8925925925925926, 'recall': 0.8970223325062034, 'f1-score': 0.8948019801980197, 'AUC': 0.926385021797354}, 'RandomForest': {'accuracy': 0.9617604617604618, 'precision': 0.9430379746835443, 'recall': 0.924317617866005, 'f1-score': 0.9335839598997494, 'AUC': 0.9507142514558915}, 'LogRegression': {'accuracy': 0.913059163059163, 'precision': 0.8598726114649682, 'recall': 0.8374689826302729, 'f1-score': 0.8485229415461973, 'AUC': 0.8907589063710877}}


In [ ]:
# result_files = os.listdir('feature_selection_results')
#result_files = list(map(lambda x: os.path.join('feature_selection_results', x), result_files))